In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from datetime import datetime

# Instalar Webdriver
service = Service(ChromeDriverManager().install())

# Abrir aba Web
driver = webdriver.Chrome(service=service)
# Localizar website e entrar
website = 'https://www.cbf.com.br/futebol-brasileiro/atletas/campeonato-brasileiro-serie-a/2024'
driver.get(website)
# Maximizar aba Web
driver.maximize_window()
# Implicit wait de 5 segundos para renderizar a página ao abrir
time.sleep(5)

# Criar listas vazias para armazenar a informação que queremos
jogador = []
anos = [2024]
competições = ['campeonato-brasileiro-serie-a','campeonato-brasileiro-serie-b','campeonato-brasileiro-serie-c','campeonato-brasileiro-serie-d','campeonato-brasileiro-feminino','campeonato-brasileiro-feminino-a1','campeonato-brasileiro-feminino-a2','campeonato-brasileiro-feminino-a3','campeonato-brasileiro-sub20','campeonato-brasileiro-aspirantes','campeonato-brasileiro-feminino-sub20','copa-brasil-masculino','copa-brasil-feminino','copa-brasil-sub20','copa-nordeste-masculino','copa-nordeste-eliminatorias','copa-nordeste-sub20','copa-verde','selecoes-estaduais-sub20','supercopa-masculino','supercopa-sub20','supercopa-feminina']

# Fechar cookies
close_cookies = driver.find_element(by='xpath', value='//button[@id="cookie-banner-lgpd-action"]')
close_cookies.click()

for ano in anos:
    for competição in competições:
        website = f'https://www.cbf.com.br/futebol-brasileiro/atletas/{competição}/{ano}'
        driver.get(website)
        while True:
            try:
                # Relocalize o elemento da tabela dentro do loop para evitar o erro StaleElementReferenceException
                container = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//table[@class="table m-t-30"]')))
                products = WebDriverWait(container, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/tr[position() > 1]')))

                for product in products:
                    jogador.append(product.find_element(by='xpath', value=".//a").text)
            except (NoSuchElementException, TimeoutException):
                print(f"Erro ao processar a página {website}. Elemento não encontrado ou tempo esgotado.")
                break

            try:
                next_button = driver.find_element(By.XPATH, '//a[@rel="next"]')
                next_button.click()
            except (NoSuchElementException, TimeoutException):
                break

valores = [i for i in range(1, 37)]
elementos_b = []

for valor in valores:
    url = f'https://www.cbf.com.br/a-cbf/arbitragem/relacao-arbitros?f=0&c=0&j=0&page={valor}'

    driver.get(url)

    try:
        # Encontre o container com os elementos <td>
        container = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="menu-panel"]/article/div/div/div/div/table'))
        )

        # Encontre todos os elementos <td> dentro do container
        td_elements = container.find_elements(By.XPATH, '//td[@scope="row"]')

        # Itere sobre os elementos <td>
        for td_element in td_elements:
            # Encontre o elemento <b> dentro do elemento <td>
            b_elements = td_element.find_elements(By.TAG_NAME, 'b')

            # Itere sobre os elementos <b> e obtenha o texto
            for b_element in b_elements:
                b_text = b_element.text
                elementos_b.append(b_text)
    except (NoSuchElementException, TimeoutException):
        print(f"Erro ao processar a página {url}. Elemento não encontrado ou tempo esgotado.")
        continue

# Imprima a lista de textos dos elementos <b>
print(elementos_b)
print(len(elementos_b))

url = 'https://www.cbf.com.br/a-cbf/diretoria/index/diretoria'
driver.get(url)

try:
    # Encontre o container com xpath //div[@class="article-inner-content"]
    container = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//div[@class="article-inner-content"]'))
    )

    # Encontre o elemento <h3> dentro de //*[@id="show-content"]/div[1]/div/div[1]
    h3_element = container.find_element(By.XPATH, '//*[@id="show-content"]/div[1]/div/div[1]/h3[1]')
    h3_text = h3_element.text

    # Encontre o elemento <p> dentro de //*[@id="show-content"]/div[1]/div/div[1]/p[2]
    p2_element = container.find_element(By.XPATH, '//*[@id="show-content"]/div[1]/div/div[1]/p[2]')
    p2_text = p2_element.text

    # Encontre todos os elementos <p> dentro do container
    p_elements = container.find_elements(By.TAG_NAME, 'p')

    # Inicialize uma lista para armazenar os textos dos elementos <p>
    p_texts = []

    # Itere sobre os elementos <p> e obtenha o texto
    for p_element in p_elements:
        p_text = p_element.text
        p_texts.append(p_text)

    # Imprima os resultados
    print(f'Texto do h3: {h3_text}')
    print(f'Texto do p2: {p2_text}')
    print(f'Texto dos elementos p (após remoção dos dois primeiros): {p_texts}')

    # Adicione todos os textos à lista chamada diretoria
    diretoria = [h3_text, p2_text] + p_texts
except (NoSuchElementException, TimeoutException):
    print(f"Erro ao processar a página {url}. Elemento não encontrado ou tempo esgotado.")
    diretoria = []

# Remova os valores específicos
valores_a_remover = [' ', 'Antônio Aquino\nFernando Sarney\nFrancisco Novelletto\nHélio Cury\nMarcus Vicente\nReinaldo Carneiro Bastos\nRoberto Góes\nRubens Lopes ']
diretoria = [item for item in diretoria if item not in valores_a_remover]

# Adicione nomes individuais à lista
if len(valores_a_remover) > 1:
    nomes = valores_a_remover[1].split('\n')
    diretoria.extend(nomes)
print(diretoria)
print(len(diretoria))

equipe = jogador + elementos_b + diretoria
equipe = list(set(equipe))

# Adicionar colunas "modalidade" e "data"
modalidade = ['futebol'] * len(equipe)
data_atual = datetime.today().strftime('%d/%m/%Y')
data = [data_atual] * len(equipe)

print(len(equipe))

driver.quit()

Erro ao processar a página https://www.cbf.com.br/futebol-brasileiro/atletas/campeonato-brasileiro-serie-d/2024. Elemento não encontrado ou tempo esgotado.
Erro ao processar a página https://www.cbf.com.br/futebol-brasileiro/atletas/campeonato-brasileiro-feminino/2024. Elemento não encontrado ou tempo esgotado.
Erro ao processar a página https://www.cbf.com.br/futebol-brasileiro/atletas/campeonato-brasileiro-aspirantes/2024. Elemento não encontrado ou tempo esgotado.
Erro ao processar a página https://www.cbf.com.br/futebol-brasileiro/atletas/copa-brasil-masculino/2024. Elemento não encontrado ou tempo esgotado.
Erro ao processar a página https://www.cbf.com.br/futebol-brasileiro/atletas/copa-brasil-feminino/2024. Elemento não encontrado ou tempo esgotado.
Erro ao processar a página https://www.cbf.com.br/futebol-brasileiro/atletas/copa-brasil-sub20/2024. Elemento não encontrado ou tempo esgotado.
Erro ao processar a página https://www.cbf.com.br/futebol-brasileiro/atletas/copa-nordest

In [2]:
df_list = pd.DataFrame({
    'Nome': equipe,
    'Modalidade': modalidade,
    'Data': data
})

df_list.to_csv('CBF_Futebol_Membros_2024.csv', index=False)